# Data Warehouse Truven QA - Member Enrollment Monthly

Performing QA on member_enrollment_monthly table in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Table Information

This table contains enrollment information on a monthly level. Depending on the data source, this information can be easily extracted in a monthly level.

Data Sources:

* Optum Zip/Optum DoD: Enrollment information not on monthly level. Enrollment dates have a begin date and an end  date which may be longer than a month. Enrollment tables are **mbr_enroll** and **mbr_co_enroll**
* Truven: Enrollment table , **t**, contains monthly level enrollment data
* Medicaid: Enrollment tables (**enrl**, **chip_uth**, **htw_enrl**) are in month level usually identified by **elig_month/elig_date** column
* Medicare: ?


Ideally we should have counts of enrollment tables from raw sources. These counts are included with the rest of the raw data tables counts for the given data sources.

* Optum Zip: **qa_reporting.optum_zip_counts**
* Optum Dod: **qa_reporting.optum_dod_counts**
* Medicaid: **qa_reporting.mdcd_enrollment_counts_[cy/fy]**
* Truven: **qa_reporting.truven_counts**
* Medicare: Needs to be created

## Row Counts and Enrollment Counts

In [30]:
query = ''' drop table if exists qa_reporting.dw_truv_mbr_enrl_monthly;
create table qa_reporting.dw_truv_mbr_enrl_monthly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    src_row_count int,
    row_count_diff int,
    row_count_diff_percentage float,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

Truven is straight forward to QA for this table due to how simple we extract the raw data and insert it into this enrollment table. We just need to see if all the rows in the ccaet and mdcrt tables are represented in this table. This means that we compare the row counts of the raw table and the member_enrollment_monthly table for each year. We also count all distinct member ids in the raw tables and in the member_enrollment_monthly table.

In [31]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_truv_mbr_enrl_monthly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        current_date
  from dw_staging.truv_member_enrollment_monthly
 group by 1,2,3;
      '''

      cursor.execute(query)

      query = '''
update qa_reporting.dw_truv_mbr_enrl_monthly a
set src_row_count = b.row_count,
    row_count_diff = a.dw_row_count - b.row_count,
    row_count_diff_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.truven_counts b
where data_source = 'truv'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)
      

After inserting the counts from the dw_staging schema, let's see if there are any years where the counts do not match with the raw tables.

In [32]:
query = '''
select * 
from qa_reporting.dw_truv_mbr_enrl_monthly
order by calendar_year
;'''
member_monthly_df = pd.read_sql(query, con=connection)
member_monthly_df


c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,truv,2011,ccaet,564688998,564688998,0,0.0,55559154,55559154,55559154,0,0.0,2023-04-17
1,truv,2011,mdcrt,56639475,56639475,0,0.0,5243029,5243029,5243029,0,0.0,2023-04-17
2,truv,2012,ccaet,567019385,567019385,0,0.0,55975628,55975628,55975628,0,0.0,2023-04-17
3,truv,2012,mdcrt,51474037,51474037,0,0.0,4874717,4874717,4874717,0,0.0,2023-04-17
4,truv,2013,ccaet,442453307,442453307,0,0.0,43737217,43737217,43737217,0,0.0,2023-04-17
5,truv,2013,mdcrt,45238684,45238684,0,0.0,4271755,4271755,4271755,0,0.0,2023-04-17
6,truv,2014,mdcrt,41158747,41158747,0,0.0,3868830,3868830,3868830,0,0.0,2023-04-17
7,truv,2014,ccaet,475186284,475186284,0,0.0,47258528,47258528,47258528,0,0.0,2023-04-17
8,truv,2015,mdcrt,24246912,24246912,0,0.0,2199633,2199633,2199633,0,0.0,2023-04-17
9,truv,2015,ccaet,289793831,289793831,0,0.0,28348363,28348363,28348363,0,0.0,2023-04-17


In [33]:
member_monthly_df[(member_monthly_df['row_count_diff'] != 0) | (member_monthly_df['mbr_count_diff'] != 0)]

,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated


If **member_monthly_df** does not have any rows, it means that all of the rows from the raw tables are in this enrollment table at a monthly level.

## Gender Count

Now that we have verified that most if not all of the rows from the raw tables, ccaet and mdcrt, have been added to the member_enrollment_monthly table, we will check that the counts for other columns such as gender have been correctly added to the DW table.

In this case we won't seperate the counts by source table, just by calendar year.

In [39]:
query = '''with truven_gen_cd as (
    select year, enrolid, sex
    from truven.ccaet
    union all
    select year, enrolid, sex
    from truven.mdcrt
),
truven_gen as (
    select year, c.gender_cd, count(*) gender_count
    from truven_gen_cd m
    left outer join reference_tables.ref_gender c
    on c.data_source = 'trv'
    and c.gender_cd_src = m.sex::text
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.truv_member_enrollment_monthly
    group by 1,2
)
select a.year, a.gender_count as dw_gender_count, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_count_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_count_diff_percentage
from truven_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd;
'''

pd.read_sql(query,  con=connection)

,year,dw_gender_count,src_gender_count,gender_count_diff,gender_count_diff_percentage
0,2018,139875408,139875408,0,0.0
1,2014,266806191,266806191,0,0.0
2,2022,59368413,59368413,0,0.0
3,2013,235317242,235317242,0,0.0
4,2020,134269786,134269786,0,0.0
5,2018,148641192,148641192,0,0.0
6,2019,133644975,133644975,0,0.0
7,2011,321553443,321553443,0,0.0
8,2015,163688497,163688497,0,0.0
9,2012,319260151,319260151,0,0.0


## Plan Type Counts

In [40]:
# Including enrollments where the plantyp column is NULL. Treating it as if unknown.
query = '''with truven_enroll as (
    select year, enrolid, case when plantyp is null then 99 else plantyp end as plantyp
    from truven.ccaet
    union all
    select year, enrolid, case when plantyp is null then 99 else plantyp end as plantyp
    from truven.mdcrt
),
truven_plans as (
    select year, case when d.plan_type is null then 'UNK' else d.plan_type end as plan_type,
            count(*) plan_count
    from truven_enroll m
    left outer join reference_tables.ref_plan_type d
    on d.data_source = 'trv'
  and d.plan_type_src::int = m.plantyp 
    group by 1,2
), dw_plans as (
    select year, case when plan_type is null then 'UNK' else plan_type end as plan_type,
            count(*) plan_count
    from dw_staging.truv_member_enrollment_monthly
    group by 1,2
)
select a.year, a.plan_type, a.plan_count as dw_plan_count, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_count_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_count_diff_percentage
from truven_plans b
full outer join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_count_df = pd.read_sql(query,  con=connection)
plan_count_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
0,2011,CMP,28277528,28277528,0,0.0
1,2011,EPO,14383668,14383668,0,0.0
2,2011,HDHP,17394920,17394920,0,0.0
3,2011,POS,39984570,39984570,0,0.0
4,2011,UNK,44773932,44773932,0,0.0
...,...,...,...,...,...,...
95,2022,EPO,942556,942556,0,0.0
96,2022,UNK,2990342,2990342,0,0.0
97,2022,PPO,53027452,53027452,0,0.0
98,2022,CDHP,12377188,12377188,0,0.0


In [41]:
plan_count_df[plan_count_df['plan_count_diff'] != 0]

,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage


In [34]:
year_row_count = member_monthly_df.groupby(['calendar_year'])[['dw_row_count', 'dw_uth_mbr_id_count']].sum()
year_row_count

,dw_row_count,dw_uth_mbr_id_count
calendar_year,,
2011,621328473,60802183
2012,618493422,60850345
2013,487691991,48008972
2014,516345031,51127358
2015,314040743,30547996
2016,320175486,30856457
2017,287386861,27620062
2018,288516600,28218353
2019,276086833,27021218


In [35]:
plan_count_df.groupby(['year'])[['dw_plan_count', 'src_plan_count']].sum()

,dw_plan_count,src_plan_count
year,,
2011,621328473,621328473
2012,618493422,618493422
2013,487691991,487691991
2014,516345031,516345031
2015,314040743,314040743
2016,320175486,320175486
2017,287386861,287386861
2018,288516600,288516600
2019,276086833,276086833


In [36]:
pd.DataFrame(year_row_count).merge(plan_count_df.groupby(['year'])[['dw_plan_count', 'src_plan_count']].sum(), how='inner', left_on='calendar_year', right_on='year')

,dw_row_count,dw_uth_mbr_id_count,dw_plan_count,src_plan_count
0,621328473,60802183,621328473,621328473
1,618493422,60850345,618493422,618493422
2,487691991,48008972,487691991,487691991
3,516345031,51127358,516345031,516345031
4,314040743,30547996,314040743,314040743
5,320175486,30856457,320175486,320175486
6,287386861,27620062,287386861,287386861
7,288516600,28218353,288516600,288516600
8,276086833,27021218,276086833,276086833
9,260249638,25009767,260249638,260249638


## Employee Status

In [43]:
query = '''with truven_enroll as (
    select year, enrolid, eestatu
    from truven.ccaet
    union all
    select year, enrolid, eestatu
    from truven.mdcrt
),
truven_plans as (
    select year, eestatu,
            count(*) employee_status_count
    from truven_enroll m
    group by 1,2
), dw_plans as (
    select year, employee_status,
            count(*) employee_status_count
    from dw_staging.truv_member_enrollment_monthly
    group by 1,2
)
select a.year, a.employee_status, a.employee_status_count as dw_employee_status_count, b.employee_status_count as src_employee_status_count, 
        a.employee_status_count - b.employee_status_count as employee_status_count_diff, 
        100. * abs(a.employee_status_count - b.employee_status_count) / b.employee_status_count as employee_status_count_diff_percentage
from truven_plans b
join dw_plans a
on a.year = b.year
and a.employee_status::int = b.eestatu
order by year;
'''

employee_status_count_df = pd.read_sql(query,  con=connection)
employee_status_count_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,employee_status,dw_employee_status_count,src_employee_status_count,employee_status_count_diff,employee_status_count_diff_percentage
0,2011,6,1873445,1873445,0,0.0
1,2011,3,14644336,14644336,0,0.0
2,2011,8,3250125,3250125,0,0.0
3,2011,9,353783891,353783891,0,0.0
4,2011,2,4566047,4566047,0,0.0
...,...,...,...,...,...,...
103,2022,4,5030627,5030627,0,0.0
104,2022,1,86734950,86734950,0,0.0
105,2022,6,418780,418780,0,0.0
106,2022,8,862231,862231,0,0.0


In [44]:
employee_status_count_df[employee_status_count_df['employee_status_count_diff'] != 0]

,year,employee_status,dw_employee_status_count,src_employee_status_count,employee_status_count_diff,employee_status_count_diff_percentage
